ETL (Extract, Transform, Load)  to pgsql

In [7]:
# Psycopg is the most popular PostgreSQL database adapter for the Python programming language
!pip install psycopg2

In [8]:
import psycopg2
import pandas as pd
import numpy as np
import re # RegEx can be used to check if a string contains the specified search pattern
import math

### Function: `connectToPgsql()` — Establish a Connection to PostgreSQL Database

This function performs the following tasks:

1. **Uses the `psycopg2` library**: It relies on the `psycopg2` library to connect to the PostgreSQL database.
2. **Takes credentials as input**: The function requires the following credentials:
   - Database name
   - User
   - Password
   - Host
   - Port
3. **Establishes a connection**: It establishes a connection to the PostgreSQL database using the provided credentials.
4. **Returns the connection object**: The function returns a connection object (`conn`) which can be used for:
   - Querying
   - Interacting with the database

In [9]:

def connectToPgsql() :
     try:
        conn = psycopg2.connect(database="osm", 
                                 user="adityadaharwal", 
                                 host="localhost", 
                                 password="1234", 
                                 port=5432)
        return conn
     except  Exception as e:
        print(f"Error: {e}")
        return None
    

### Query and Display PostgreSQL Extensions in a DataFrame

This code snippet performs the following steps:

1. **Connects to the PostgreSQL Database**: It establishes a connection to the PostgreSQL database using appropriate credentials.
   
2. **Retrieves Installed Extensions**: It queries the `pg_extension` system catalog to fetch information about the extensions installed in the database.

3. **Displays Results in a Pandas DataFrame**: The retrieved data is displayed in a Pandas DataFrame for a structured and easy-to-read format.

   - This allows a quick overview of the PostgreSQL extensions installed in the current database instance.

In [10]:
conn = connectToPgsql()

# show all type of dat in database
cur = conn.cursor()
cur.execute('SELECT * FROM pg_extension;')

# Fetch results
data = cur.fetchall()

# Get column names
colnames = [desc[0] for desc in cur.description]

# Create a DataFrame
df_extension = pd.DataFrame(data, columns=colnames)

conn.commit()
conn.close()

print(df_extension)

     oid  extname  extowner  extnamespace  extrelocatable extversion  \
0  13826  plpgsql        10            11           False        1.0   
1  96395  postgis     16384          2200           False      3.4.2   
2  97471   hstore     16384          2200            True        1.8   

  extconfig                                       extcondition  
0      None                                               None  
1   [96713]  [WHERE NOT (\nsrid BETWEEN 2000 AND 2217\nOR s...  
2      None                                               None  


## Creating and Indexing the `features` Table in PostgreSQL

This script performs the following tasks:

1. **Creates the `features` Table**: 
   - A table is created to store various types of geographic features.
   - The table includes a `geom` column of the `GEOGRAPHY` data type to store geographic data.

2. **Sets Up a GiST Index**: 
   - The `CREATE INDEX` statement is used to create a GiST (Generalized Search Tree) index on the `geom` column.
   - This index optimizes spatial queries, such as finding geographic features near a specific point or performing range queries.

By creating the index, the performance of spatial queries is improved, making geographic data analysis more efficient.

In [ ]:
conn = connectToPgsql()

# create table
cur = conn.cursor()

# tag type:
# college 
# govt_institute
# malls 
# schools
# luxury_hotel
# prime_hotel
# hotel
# it_company 
# industry
# tourist_spot


# create table
cur.execute("""CREATE TABLE  features (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    review_count INTEGER,
    rating NUMERIC(2, 1), 
    tag VARCHAR(255) NOT NULL,
    geom GEOGRAPHY(Point, 4326)
    );
""")

# create indexing on table
cur.execute('CREATE INDEX idx_features_geom ON features USING GIST (geom);')


# commit the transaction and close the connection
conn.commit()
conn.close()

### Function: `extract_lat_long(url)`

This function is a helper designed to:

1. **Extract Latitude and Longitude**: It takes a URL as input and extracts the latitude and longitude information embedded within the URL.
   
2. **Return Coordinates**: The function returns the extracted latitude and longitude as separate values (or as a tuple, depending on implementation). 



In [25]:
def extract_lat_long(url):
    # Regular expression to match latitude and longitude
    pattern =  r"3d([\d.-]+)!4d([\d.-]+)"
    match = re.search(pattern, url)

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return [latitude, longitude]
    else:
        return None, None

### Inserting Features from CSV into PostgreSQL Database

This function performs the following tasks:

1. **Processes a CSV File**: 
   - It takes a CSV file containing geographical feature data as input.
   - The CSV file may include columns like title, reviews, rating, and coordinates.

2. **Cleans and Formats the Data**:
   - The function cleans and formats the data as needed, ensuring that the information is in the correct format before insertion.

3. **Extracts Coordinates from Google Maps Links**:
   - Coordinates (latitude and longitude) are extracted from Google Maps links present in the CSV file.

4. **Inserts Data into PostgreSQL Database**:
   - The formatted and cleaned data, including title, reviews, rating, and coordinates, is inserted into the PostgreSQL database.

---

### Function: `push_features_in_DB(file_path, tag)`

This function performs the following:

1. **Takes Input Parameters**:
   - `file_path`: Path to the CSV file containing the data.
   - `tag`: A tag or identifier used for categorizing or filtering the data during the insertion process.

2. **Inserts Data into the Database**:
   - It reads the CSV file, processes each entry, and inserts the features into the PostgreSQL database.

In [ ]:

def push_features_in_DB(file_path,tag):
    feature = pd.read_csv(file_path)
    feature = feature.drop(columns=['Website','Phone','Industry','Address'])
    feature['Google Maps Link'] = feature['Google Maps Link'].apply(extract_lat_long)
    feature['Google Maps Link'].head()
    feature.rename(columns={'Google Maps Link': 'Coordinates'}, inplace=True)
    # feature.head()

    #remove NAN 
    feature.dropna(subset=['Title', 'Rating','Reviews'])
    
    conn = connectToPgsql()
    cur = conn.cursor()
    
    # Loop through the data and insert each row
    for row in feature.itertuples():
        
        Title = row.Title
        Rating = row.Rating
        Reviews = row.Reviews
        Coordinates = row.Coordinates
        
        if pd.isna(Reviews) :
            Reviews = '0'
        if pd.isna(Rating) :
            Rating = 0.0

        Title = Title.replace('\'', '')
        if isinstance(Reviews, str):
            Reviews = Reviews.replace(',', '')
            Reviews = int(Reviews)

        query = f"""
            INSERT INTO features (title, review_count, rating, tag, geom) 
            VALUES ('{Title}',{Reviews}, {Rating},'{tag}' , ST_SetSRID(ST_MakePoint({Coordinates[1]}, {Coordinates[0]}), 4326));
        """
        cur.execute(query)

    conn.commit()
    conn.close()
    

### Inserting Multiple Feature Datasets into PostgreSQL Database

This section outlines the process for inserting data from multiple CSV files into a PostgreSQL database. The steps are as follows:

1. **Handling Multiple CSV Files**: 
   - Multiple CSV files, each containing different types of geographical features, are processed.

2. **Using the `push_features_in_DB` Function**: 
   - The `push_features_in_DB` function is called for each CSV file.
   - The function is passed the path to the CSV file and an appropriate tag that identifies the type of feature being inserted (e.g., park, restaurant, landmark).

3. **Inserting Data into the Database**: 
   - For each CSV file, the function processes the data and inserts it into the PostgreSQL database with the corresponding tag.

This approach allows you to efficiently insert and categorize different datasets of geographical features into a single database.

In [ ]:
push_features_in_DB('../Datasets/college.csv','college')
push_features_in_DB('../Datasets/govt_institutes.csv','govt_institute')
push_features_in_DB('../Datasets/hotel.csv','hotel')
push_features_in_DB('../Datasets/luxury_hotel.csv','luxury_hotel')
push_features_in_DB('../Datasets/prime_hotel.csv','prime_hotel')
push_features_in_DB('../Datasets/malls.csv','malls')
push_features_in_DB('../Datasets/schools.csv','schools')
push_features_in_DB('../Datasets/it_companies.csv','it_companies')
push_features_in_DB('../Datasets/tourist_spot.csv','tourist_spot')
push_features_in_DB('../Datasets/hospital.csv','hospital')

### For Testing Purpose: Delete the `features` Table

In [14]:
# conn = connectToPgsql()

# # create table
# cur = conn.cursor()
# cur.execute("Drop TABLE features;")
# conn.commit()
# conn.close()